<img src="escudo_utfsm.gif" style="float:right;height:100px">
<img src="IsotipoDIisocolor.png" style="float:left;height:100px">
<center>
    <h1> ILI286 - Computación Científica II</h1>
    <h1> Tarea 1: Deflation y Matrices Sparse </h1> 
    <h3> [S]cientific [C]omputing [T]eam 2019</h3>
    <h3> Guillermo Godoy / 201391001-7</h3>
</center>
<p>
<center>Agosto 2019 - v1.0 </center>
</p>

---

In [137]:
import numpy as np
import scipy as sp
import scipy.sparse as sparse
from time import time
from scipy import sparse
import sys
from scipy.sparse import lil_matrix
from scipy.sparse import dok_matrix
from scipy.sparse import csr_matrix
import datetime


## Contexto

Esta tarea tiene como objetivo aplicar los contenidos vistos en clase, respecto a métodos numéricos para obtener valores y vectores propios. Además, esta instancia servirá para recordar (o aprender) el uso de *Jupyter Notebook*, junto con las principales bibliotecas del curso: *Numpy* y *Scipy*.

## Sección 1: Algoritmo Deflation (30 puntos)

Se discutió en clases que una de las principales propiedades que tienen las matrices simétricas es que

* Sus valores propios son reales.
* Sus vectores propios son ortogonales.

Sabemos que por medio de *Power Iteration* e *Inverse Power Iteration*, podemos rescatar algunos de los valores propios y vectores propios. Mediante *Normalized Simultaneous Iteration* o *Unshifted QR*, podemos extraer todos los valores y vectores propios al mismo tiempo. Ahora veremos un método distinto para extraer los valores y vectores propios, uno por uno.

Consideremos una matriz $A \in \mathbb{R}^{n \times n}$ simétrica cuyos valores propios son $\lambda_i$ (por simplicidad, consideraremos que también son todos positivos) y sus vectores propios son $\mathbf{v}_i$, para $i \in \{1, 2, \dots, n\}$. Considere además que $\lambda_i > \lambda_j$, para todo $i < j$, es decir, los valores propios están ordenados. Considere ahora la matriz $A_1$ definida como:

$$
    A_1 = A - \lambda_1\mathbf{v}_1\,\mathbf{v}_1^T.
$$

**1. (5 puntos)** Demuestre que los vectores propios de $A$ también son los vectores propios de $A_1$. ¿Cuáles son los valores propios de $A_1$?

R:
Supuesto: Los vectores propios son $\mathbf{v}_i$, para $i \in \{1, 2, \dots, n\}$, solo consideraremos los vectores normalizados.

Tenemos por propiedad: 
$$
\begin{align}
A\mathbf{v}_i & = \lambda_i\mathbf{v}_i \\ 
A\mathbf{v}_1 & = \lambda_1\mathbf{v}_1 \\ 
A_1\mathbf{v}_{A_1} & = \lambda_{A_1}\mathbf{v}_{A_1}
\end{align}
$$
Donde tenemos que obtener:
$$
\begin{align}
\mathbf{v}_{A_1} & = \mathbf{v}_i
\end{align}
$$
Modificamos la Matriz $A_1$ dada anteriormente de la siguiente forma:
$$
\begin{align}
A_1 &= A - \lambda_1\mathbf{v}_1\,\mathbf{v}_1^T \\ 
A_1\mathbf{v}_i&=A\mathbf{v}_i-A\mathbf{v}_1\,\mathbf{v}_1^T\mathbf{v}_i \\
\end{align}
$$
Tenemos 2 casos:

Con $i\neq 1 $ (Dado que $A=A^T$ y $\lambda_i > \lambda_j$ entonces $$v_1^Tv_i=0$$ (Vectores propios con valores propios distintos son ortogonales))
$$
\begin{align}
A_1\mathbf{v}_i&=A\mathbf{v}_i-A\mathbf{v}_1\,\mathbf{v}_1^T\mathbf{v}_i \\
A_1\mathbf{v}_i&=A\mathbf{v}_i\\
A_1\mathbf{v}_i&=\lambda_i\mathbf{v}_i
\end{align}
$$
Con $i=1$
$$
\begin{align}
A_1\mathbf{v}_1&=A\mathbf{v}_1-A\mathbf{v}_1\,\mathbf{v}_1^T\mathbf{v}_1 \\
A_1\mathbf{v}_1&=A\mathbf{v}_1(1-\mathbf{v}_1^T\mathbf{v}_1)\\
\mathbf{v}_1^T\mathbf{v}_1 &= \parallel \mathbf{v}_1\parallel_2^2\\
\parallel \mathbf{v}_1\parallel_2^2&= 1\\
A_1\mathbf{v}_1&=0\mathbf{v}_1
\end{align}
$$
Por lo que obtenemos que $A_1$ tiene los mismos vectores propios de $A$ y tiene los mismos valores propios a excepción del primero, donde el primer valor propio de $A_1$ es cero

**2. (10 puntos)** El proceso anterior, denominado *Deflation*, puede repetirse iterativamente para construir un algoritmo iterativo que permita encontrar los valores y vectores propios en sucesión. El algoritmo, paso a paso, puede definirse como:

1. Se define $A_0 = A$, $\lambda_0 = 0$ y $\mathbf{v}_0 = \mathbf{0}$.
2. Para $i \in \{1, 2, \dots, n\}$
    1. Se define $A_i = A_{i-1} - \lambda_{i-1}\mathbf{v}_{i-1}\,\mathbf{v}_{i-1}^T$.
    2. Se encuentra el valor y vector propio dominante de $A_{i}$, denotados por $\lambda_{i}$ y $\mathbf{v}_{i}$, respectivamente.
3. Se retorna el conjunto $\{\lambda_1, \lambda_2, \dots, \lambda_n\}$ y $\{ \mathbf{v}_{1}, \mathbf{v}_{2}, \dots, \mathbf{v}_{n}\}$.

Implemente la función `deflation`, el cual aplicará el algoritmo *Deflation* mediante el uso de *Power Iteration*. Esta función debe recibir como parámetros:
* La matriz $A$.
* Un *initial guess* $\mathbf{x}_0$.
* Un número de iteraciones.

La función debe retornar una lista con los valores propios encontrados y una lista de los vectores propios encontrados.

In [2]:

'''
Program 12.1 Power iteration
Computes dominant eigenvector of square matrix
Input: matrix A, initial (nonzero) vector x, number of steps k
Output: dominant eigenvalue lam, eigenvector u
'''
def power_iteration(A, x, k, verbose=False):
   
    for j in range(k):
        u = x/np.linalg.norm(x)
        x = np.dot(A, u)
    lam = np.dot(u, x)
    u = x/np.linalg.norm(x)
    return (lam, u)


'''
Compute all the eigenvalues and eigenvectors using Deflation procedure.
Input:
A - (n x n matrix) Matrix to compute its eigenvalues and eigenvectors
x0 - (array) Initial guess for Power Iteration
it - (integer) Number of iteration for Power Iteration.
Output:
eig_values - (list) A list with the eigenvalues found by the algorithm
eig_vectors - (list) A list with the eigenvectors found by the algorithm
'''
def deflation(A, x0, it):
    eig_values = []
    eig_vectors = []
    lam=0
    n = A.shape[0]
    #v = np.zeros((n,1))
    v=0
    for i in range(n):
        #A -= lam*np.dot(v,v.T)
        A -= lam*v*np.transpose([v])
        lam,v = power_iteration(A, x0, it)
        eig_values.append(lam)
        eig_vectors.append(v)
    return eig_values, eig_vectors

Para probar su algoritmo, utilice la función `generate_matrix` que construye una matriz simétrica a partir de una lista de valores propios específicos.

In [80]:
'''
Given a list of n real vaues, returns a symmetric matrix whose eigenvalues are the given values.
Input:
eig_values - (list) list of n real values that will be used as eigenvalues
Output: 
A - (n x n matrix) A symmetric matrix whose eigenvalues equals the values in eig_values list
Q - (n x n matrix) An orthonormal matrix whose columns are the eigenvectors
eig_values (list) A list with eigenvalues of A. It is the same list given as input.
'''
def generate_matrix(eig_values):
    n = len(eig_values)
    Q, _ = np.linalg.qr(np.random.rand(n, n))
    L = np.diag(eig_values)
    A = np.dot(Q, np.dot(L, Q.T))
    return A, Q, eig_values
eig_values1 = []
eig_values2 = []
for i in range(10):
    eig_values1.append(i+1)
A1,Q1,eig_values1=generate_matrix(eig_values1)

for i in range(12):
    eig_values2.append(2**(i+1))
A2,Q2,eig_values2=generate_matrix(eig_values2)
print (eig_values1)
print (eig_values2)

x01=np.ones(A1.shape[0])
values1,vectores1=deflation(A1, x01, 1000)
print(values1)

x02=np.ones(A2.shape[0])
values2,vectores2=deflation(A1, x01, 1000)
print(values2)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]
[9.999999999999998, 9.0, 8.0, 7.0000000000000036, 6.0, 5.0000000000000036, 3.999999999999999, 3.000000000000001, 1.9999999999999996, 1.0000000000000004]
[1.0000000000000004, -1.8846865067248437e-15, -1.6302029670098922e-15, -1.1324692493066439e-15, -8.730118902266693e-16, -4.3186236596426137e-16, -7.812894579662378e-17, -7.327850282731556e-17, 5.533214947236848e-17, -8.311585499942047e-17]


Genere matrices con los siguientes valores propios:
* $\{1, 2, \dots, 10\}$.
* $\{2, 2^2, 2^3, \dots, 2^{12}\}$.

Utilice las matrices generadas para probar su algoritmo. Su tarea será probada con otras matrices, pero puede utilizar estas para verificar la correctitud de su algoritmo.

**3. (15 puntos)** Implemente el algoritmo *Normalized Simultaneous Iteration* (NSI). Genere matrices de los siguientes tamaños, utilizando la función `generate_matrix`:

* $100 \times 100$.
* $1000 \times 1000$.
* $10000 \times 10000$.
* $15000 \times 15000$.

Estime teóricamente cuanta memoria requiere cada uno de los algoritmos. Además, ejecute ambos algoritmos sobre las matrices generadas y mida el tiempo de ejecución. Concluya respecto a los resultados.

In [87]:
def NSI(A, k):
    Q = np.identity(A.shape[0])
    for i in range(k):
        Q, R = np.linalg.qr(np.dot(A, Q))
    return (np.diag(np.dot(np.dot(Q.transpose(),np.diag(A)),Q)))

for i in range(15000):
    eig_values1.append(i+1)
A100,Q1,_=generate_matrix(eig_values1[:100])
print("A100",A100.shape[0])

A1000,Q1,_=generate_matrix(eig_values1[:1000])
print("A1000",A1000.shape[0])
A10000,Q1,_=generate_matrix(eig_values1[:10000])
print("A10000",A10000.shape[0])
#A15000,Q1,_=generate_matrix(eig_values1)
#print("A15000",A15000.shape[0])

def revisar(A,k):
    print("Matriz de: ",A.shape[0],"x",A.shape[0])
    
    x0=np.ones(A.shape[0])
    ti = time()
    values1,vectores1=deflation(A, x0, k)
    tf = time()
    tiempo = tf-ti
    print("Tiempo: ",str(datetime.timedelta(seconds=tiempo))," para deflation con ",k," iteraciones")
    ti = time()
    values1=NSI(A, k)
    tf = time()
    tiempo = tf-ti
    print("Tiempo: ",str(datetime.timedelta(seconds=tiempo))," para NSI con ",k," iteraciones")

revisar(A100,100)
revisar(A1000,100)
#revisar(A10000,100)
#revisar(A15000,100)

A100 100
A1000 1000
A10000 10000
Matriz de:  100 x 100
Tiempo:  0:00:00.093758  para deflation con  100  iteraciones
Tiempo:  0:00:00.078132  para NSI con  100  iteraciones
Matriz de:  1000 x 1000
Tiempo:  0:00:24.329952  para deflation con  100  iteraciones
Tiempo:  0:00:13.501013  para NSI con  100  iteraciones
Matriz de:  10000 x 10000


KeyboardInterrupt: 

Teoricamente:
deflation: ocupa la matriz n * n ($n^2$) + los 2 vectores (eig_values y eig_vectors) el vector eig_values es de n y el eig_vectors es de n * n ($n^2$) dado que en cada iteracion guarda el vector propio asociado a cada valor propio. En total es
$(n^2 + n + n^2)*8[B]$ . Nota: el total es la max capacidad de memoria que puede ocupar dado que los vectores eig_values y eig_vectors, solo van a llegar a su max capacidad en la ultima iteracion.

NSI: ocupa la matriz n * n ($n^2$) y hace una desconpocicion QR de una matiz cuadrada dando Q y R del mismo tamaño n * n.
En total es $3(n^2)*8[B]$ el cual es constante (el tamaño de memoria a utilizar) en toda la ejeccion.

En tiempo de ejecucion NSI se demora menos que deflation, esto se debe principalmente a que en deflation hacemos el calculo de power_iteration, solo para obtener el mayor valor propio de la matriz que estamos trabajando en ese momento, y luego tenemos que generar una nueva matriz resultando de un calculo con la anterior y el resultado obtenido, por ende tenemos que estar calculando muchos valores preopios de "distintas" matrizes para lograr obtener todos los valores propios de la matriz original. En cambio en NSI se trabaja sobre la misma matriz y solo hay que generar una descompocicion QR sobre la matriz original y el Q anterior, estimamos más eficientemente la matriz diagonal que contiene los valores propios.
NOTA: Solo se pudo evaluar con 100x100 y 1000x1000. Ya con 10000 se demoro demaciado en responder, lo mismo al generar la matriz de 15000x15000

## Sección 2: Uso de matrices *Sparse* (60 puntos)

Cuando una matriz es de tipo *Sparse*[1], es posible trabajar sobre matrices de mayor tamaño, debido a que se evita el almacenamiento de valores *innecesarios* (en este caso, los ceros).

**1. (5 puntos)** Implemente la función `read_sparse_matrix` que reciba la ruta a un archivo de texto plano que contiene una matriz *sparse*, y construya la matriz respectiva utilizando una estructura de datos especializada para matrices *sparse*[2]. El formato del archivo será:
* La primera línea del archivo contendrá un solo número `N`, donde `N` indica el tamaño de la matriz (matriz de tamaño `N`$\times$`N`).
* Cada una de las siguientes líneas contiene 3 números separados por espacios, que indicarán las entradas no nulas de la matriz. Si la línea contiene los números `i j a`, esto indica que en la fila `i` y columna `j`, la matriz tiene un coeficiente con valor `a`.

In [128]:

'''
Read a file in the specified format to build a Sparse Matrix.
Input:
path - (string) A path to a file.
Output:
A - (N x N matrix) The matrix in a sparse format.
'''
def read_sparse_matrix(path):
    txt = open(path)
    sw=False
    for line in txt.readlines():
        if (sw):
            #print(line)
            Datos=line.strip().split()
            i=int(Datos[0])
            j=int(Datos[1])
            value=float(Datos[2])
            A[i, j]=value
        else:
            N=int(line)
            print("Matriz de",N)
            A = lil_matrix((N, N))
            #A = dok_matrix((N, N))
            #A = csr_matrix((N, N))
            sw=True
    txt.close()
    return A

M10000=read_sparse_matrix("matrix_10000.txt")
size = M10000.data.nbytes
print('Al ser sparse se guarda en memoria {:.2f} MB '.format(size / 1024 ** 2))
print('Si no fuera sparse se guardarian {:.2f}MB'.format((8*10000**2) / 1024 ** 2))
M20000=read_sparse_matrix("matrix_20000.txt")
size = M20000.data.nbytes
print('Al ser sparse se guarda en memoria {:.2f} MB '.format(size / 1024 ** 2))
print('Si no fuera sparse se guardarian {:.2f}MB'.format((8*20000**2) / 1024 ** 2))
M50000=read_sparse_matrix("matrix_50000.txt")
size = M50000.data.nbytes
print('Al ser sparse se guarda en memoria {:.2f} MB '.format(size / 1024 ** 2))
print('Si no fuera sparse se guardarian {:.2f}MB'.format((8*50000**2) / 1024 ** 2))
print('Nota: hay diferencia en memoria al utilizar distintos metodos de creacion de matrices sparse, en mi caso utilize lil_matrix')

Matriz de 10000
Al ser sparse se guarda en memoria 0.08 MB 
Si no fuera sparse se guardarian 762.94MB
Matriz de 20000
Al ser sparse se guarda en memoria 0.15 MB 
Si no fuera sparse se guardarian 3051.76MB
Matriz de 50000
Al ser sparse se guarda en memoria 0.38 MB 
Si no fuera sparse se guardarian 19073.49MB
Nota: hay diferencia en memoria al utilizar distintos metodos de creacion de matrices sparse, en mi caso utilize lil_matrix


Cargue las matrices adjuntas en los archivos `matrix_10000.txt`, `matrix_20000.txt` y `matrix_50000.txt`, de dimensiones $10000 \times 10000$, $20000 \times 20000$ y $50000\times50000$ respectivamente, utlizando la función `read_sparse_matrix`. Considerando *double precision* para los números de punto flotante, indique cuánta memoria utilizarían estas matrices si fuesen cargadas utilizando arreglos bidimensionales o estructuras no-especializadas para matrices *sparse*. ¿Existe alguna diferencia al utilizar las estructuras de datos *sparse*?

R: Los datos se muestran en el bloque anterior, hay una gran diferencia ya que las matrices con estructuras de datos *sparse*, solo alamacena los datos no nulos por ende se obtiene una mejora de esta.

**2. (25 puntos)** Suponga ahora que la matriz simétrica $A$ en el algoritmo de *Deflation* es *Sparse*. Notar que luego de calcular el valor y vector propio dominante de $A$, el resultado obtenido para calcular $A_1$:

$$
    A_1 = A - \lambda_1\mathbf{v}_1\,\mathbf{v}_1^T,
$$

**no** necesariamente es una matriz *sparse*. Esto se debe a que la matriz $\mathbf{v}_1\mathbf{v}_1^T$ no necesariamente es *sparse* y al restarse con $A$, esta propiedad puede perderse.

A pesar de lo anterior, una propiedad importante de *Power Iteration* es que no opera direcamente sobre la matriz, si no que solo es requerida a través del producto **matriz-vector**. Notar que si se desea calcular $A_1\mathbf{u}$, donde $\mathbf{u}$ es un vector arbitrario, puede calcularse como

$$
    A_1\mathbf{u} 
    = \left(A - \lambda_1\mathbf{v}_1\,\mathbf{v}_1^T\right)\mathbf{u}
    = A\mathbf{u} - \lambda_1 \mathbf{v}_1 \left(\mathbf{v}_1^T\mathbf{u}\right).
$$

De esta manera, no es necesario construir explícitamente la matriz $\mathbf{v}_1\mathbf{v}_1^T$. De manera general, la matriz $A_i\mathbf{u}$ puede calcularse como

$$
    A_i\mathbf{u} = A\mathbf{u} - \sum_{k = 1}^{i}\lambda_k \left( \mathbf{v}_k^T\mathbf{u} \right) \mathbf{v}_k.
$$

Implemente la función `A_times_vector` que reciba como parámetros: 

* Una matriz de tipo *sparse*.
* Un entero $k$.
* Una lista con $k$ valores propios.
* Una lista con $k$ vectores propios.
* Un vector $\mathbf{u}$, arbitrario.

La función debe retornar el resultado numérico del producto matriz-vector $A_{k}\mathbf{u}$.

In [129]:
'''
Implementation of a Deflation matriz-vector product to avoid building non-sparse matrix
Input:
A - (N x N matrix) A matrix in a sparse format.
k - (integer) An integer.
k_eig_values - (list) A list with k eigenvalues of A.
k_eig_vectors - (list) A list with k eigenvectors of A.
u - (array) An arbitrary vector
Output:
b - (array) The result of a Deflation matrix times u, using the given eigenvalues and eigenvectors.
'''
def A_times_vector(A, k, k_eig_values, k_eig_vectors, u):
    suma = np.zeros(A.shape[0])
    for i in range(k):
        suma +=k_eig_values[i]*np.dot(k_eig_vectors[i].T,u)*k_eig_vectors[i]
    if k!=0:
        b = A*u-suma
    else:
        b = A*u
    # b is the result of the matrix-vector product
    return b


**3. (30 puntos)** Implemente la función `sparse_deflation`, modificando su implementación inicial de *Deflation*, que permita operar sobre matrices de gran dimenensión y que sean *sparse*. Para esto, utilice la implementación realizada de la función `A_times_vector` en combinación con el algoritmo de *Power Iteration*. La función `sparse_deflation` recibe como parámetros:
* La matriz $A$ de tipo *sparse*.
* El *initial guess* $\mathbf{x}_0$.
* El número de iteraciones a realizar en *Power Iteration*
* Un número $T$ que indique el número de valores y vectores propios que se desean obtener de la matriz $A$.

In [130]:
def power_iteration2(A, x, it,k,k_eig_values, k_eig_vectors,verbose=False):
   
    for j in range(it):
        u = x/np.linalg.norm(x)
        x = A_times_vector(A, k, k_eig_values, k_eig_vectors, u)
    lam = np.dot(u, x)
    u = x/np.linalg.norm(x)
    return (lam, u)
'''
Compute T eigenvalues and eigenvectors from the sparse matrix A, using Deflation.
Input:
A - (n x n matrix) Sparse matrix
x0 - (array) Initial guess for Power Iteration
it - (integer) Number of iterations for Power Iteration
T - (integer) Number of eigenvalues and eigenvectors to compute from A
Output:
eig_values - (list) List of T eigenvalues found by the algorithm
eig_vectors - (list) List of T eigenvectors found by the algorithm
'''
def sparse_deflation(A, x0, it, T):
    eig_values = []
    eig_vectors = []
    lam = 0
    v = 0
    for k in range(T):
        lam,v = power_iteration2(A, x0, it,k,eig_values,eig_vectors)
        eig_values.append(lam)
        eig_vectors.append(v)
    return eig_values, eig_vectors

Aplique la función `sparse_deflation` sobre las matrices cargadas desde los archivos adjuntos, para distintos valores de $T$ y de iteraciones para *Power Iteration*. Mida los tiempos de ejecución y concluya al respecto. ¿Podría haber aplicado el algoritmo NSI para estas matrices? Justifique y concluya al respecto.

In [136]:
#x03=np.ones(M10000.shape[0])
#print(M10000.shape[0])
#valores3,vectores3=sparse_deflation(M10000,x03,10,10)
#print(valores3)
def revisar_sparse_deflation(A,it,T):
    print("Matriz de: ",A.shape[0],"x",A.shape[0],"sparse")
    x0=np.ones(A.shape[0])
    ti = time()
    values1,vectores1=sparse_deflation(A, x0, it,T)
    tf = time()
    tiempo = tf-ti
    print("Tiempo: ",str(datetime.timedelta(seconds=tiempo))," para sparse_deflation con ",it," iteraciones de Power Iteration y ",T,"valores distnitos")

revisar_sparse_deflation(M10000,10,10)
revisar_sparse_deflation(M10000,10,20)
revisar_sparse_deflation(M10000,20,10)
revisar_sparse_deflation(M10000,30,10)
revisar_sparse_deflation(M10000,10,30)

revisar_sparse_deflation(M20000,10,10)
revisar_sparse_deflation(M20000,10,20)
revisar_sparse_deflation(M20000,20,10)
revisar_sparse_deflation(M20000,30,10)
revisar_sparse_deflation(M20000,10,30)

revisar_sparse_deflation(M50000,10,10)
revisar_sparse_deflation(M50000,10,20)
revisar_sparse_deflation(M50000,20,10)
revisar_sparse_deflation(M50000,30,10)
revisar_sparse_deflation(M50000,10,30)
#ti = time()
#values1=NSI(M10000,10)
#tf = time()
#tiempo = tf-ti
#print("Tiempo: ",str(datetime.timedelta(seconds=tiempo))," para NSI con ",10," iteraciones")

Matriz de:  10000 x 10000 sparse
Tiempo:  0:00:16.923438  para sparse_deflation con  10  iteraciones de Power Iteration y  10 valores distnitos
Matriz de:  10000 x 10000 sparse
Tiempo:  0:00:33.315545  para sparse_deflation con  10  iteraciones de Power Iteration y  20 valores distnitos
Matriz de:  10000 x 10000 sparse
Tiempo:  0:00:33.729372  para sparse_deflation con  20  iteraciones de Power Iteration y  10 valores distnitos
Matriz de:  10000 x 10000 sparse
Tiempo:  0:00:50.594513  para sparse_deflation con  30  iteraciones de Power Iteration y  10 valores distnitos
Matriz de:  10000 x 10000 sparse
Tiempo:  0:00:54.629518  para sparse_deflation con  10  iteraciones de Power Iteration y  30 valores distnitos
Matriz de:  20000 x 20000 sparse
Tiempo:  0:00:13.696574  para sparse_deflation con  10  iteraciones de Power Iteration y  10 valores distnitos
Matriz de:  20000 x 20000 sparse
Tiempo:  0:00:30.146547  para sparse_deflation con  10  iteraciones de Power Iteration y  20 valores di

No se logro ejecutar NSI(M10000,10) hasta el final, utilizando 11 gb de memoria en el proceso, esto se debe al calculo ineficiente que hace qr y posiblemente se pierda la propiedad sparse de la matiz en el proceso.

Los tiempos mostrados en el bloque anterior, se puede concluir que no tienen mucha diferencia en el tamaño de la matriz, esto se debe a que al ser sparse solo se hace el calculo con los valores no nulos de la matriz, por ende depende de la cantidad de estos valores que tiene cada matriz.

No se encontro mucha diferencia entre aumentar el valor de $it$ (iteraciones del Power Iteration) vs aumentar solo el de la cantidad de valores propios, como se puede ver al utilizar una combinacion de 10-30 o 30-10 donde encontrar más valores propios pero menos precisos es solo un poco mas costoso que tener menos valores pero más precisos. Por ende buscar más valores es mas costoso que se más eficiente en la busqueda de estos. Pero si se quiere preciosion y cantidad el costo sube por ambos lados.


---
# Instrucciones:

* **Importante, Asegúrese de responder TODO lo que la pregunta pide.**
* La estructura de la tarea es la siguiente:
     1. Título, nombre de estudiante, email y rol.
     2. Responder cada pregunta de forma personal.
     5. Referencias. Es muy importante incluir todas las fuentes usadas, de otra forma se considera que lo no se ha citado adecuadamente es su trabajo.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`) entregado.
* Recuerde responder la encuesta en el plazo establecido
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, `Matplotlib` y `ipywidgets`, entre otras, así como la **correcta implementación de algoritmos vectorizados**.
* **MUY IMPORTANTE** El archivo de entrega debe denominarse TareaN-rol.tar.gz y _notebook_ debe tener como nombre TareaN-rol.ipynb, donde $N$ es el número de la tarea y debe contener un directorio con todos los archivos necesarios para ejecutar el notebook, junto con un archivo README indicando explícitamente las librerías o módulos utilizados, nombre y rol del estudiante. Por cada error en este ambito implicará un descuento de 30 puntos.
* El descuento por día de atraso será de $30$ puntos, con un máximo de 1 día de atraso. No se recibirán entregas después de este día.
* Debe citar toda fuente de código externo. 
* El trabajo es personal, no se permite compartir código ni utilizar código de otros, aunque sí se sugiere discutir aspectos generales con sus compañeros.
* En caso de sospecha de no cumplimiento de estas instrucciones, se solicitará al involucrado o la involucrada a aclarar la situación. Dependiendo de la justificación se decidirá su calificación, la cual podrá o no ser penalizada.
* El no seguir estas instrucciones, implica descuentos en su nota obtenida.

---

# Referencias


[1] Sparse matrix, https://en.wikipedia.org/wiki/Sparse_matrix

[2] Sparse module, Scipy. https://docs.scipy.org/doc/scipy/reference/sparse.html

https://github.com/tclaudioe/Scientific-Computing/blob/master/SC2/U1_EigenWorld.ipynb